In [34]:
#USER INPUT HERE
modweight = 0.7 #change this value to the weight we use for modified vs avg epv

#change the name of the filename being used as the mod_epv for the current matchup
filename = 'mod_epv_clemson' #don't include the file extension

mod_epv = __import__(filename)
from mod_epv import mod_epv as m_epv

teamname = filename.split('_')[2] 

print('opponent = ', teamname)

opponent =  clemson


In [35]:
#import statements
import numpy as np
import pandas as pd
#these two imports will always be the same:
from avg_epv import avg_epv as a_epv
from r_conv_pct import conv_pct as r_conv

In [36]:
r_conv = np.matrix(r_conv)

In [37]:
#fn for ceiling division
def ceildiv(a, b):
    return -(-a // b)

#fn for grouping rows into bins of size binsize and taking the average of each bin
def binaverage(matrix, binsize):
    lst = np.array(matrix)
    output = np.zeros((len(lst), (ceildiv(len(lst[0]),binsize))))
    for i in range(0, len(lst)):
        temp = []
        for j in range(0, len(lst[0]), binsize):
            temp2 = lst[i][j:j + binsize]
            temp2 = [x for x in temp2 if x is not None]
            temp.append(sum(temp2) / len(temp2))
        output[i] = np.array(temp)
    return output

In [46]:
#output the average conversion percentage independent of yardline100 bin
print('conversionbydist =\n', binaverage(r_conv, 10))
#not necessary but cool to look at if we want to use this for something

conversionbydist =
 [[0.724]
 [0.614]
 [0.532]
 [0.473]
 [0.432]
 [0.405]
 [0.386]
 [0.371]
 [0.355]]


In [39]:
#place EPVs into 10 yard bins to match conversion percentages
a_epv10 = binaverage(a_epv, 10)
m_epv10 = binaverage(m_epv, 10)

In [40]:
#print 10 yard bin averages
np.set_printoptions(precision=3, suppress = True)
print(a_epv10, '\n')
print(m_epv10)

[[ 4.063  3.261  2.714  2.166  1.619  1.072  0.525 -0.022 -0.569 -0.87 ]
 [ 4.118  3.315  2.768  2.221  1.674  1.127  0.58   0.033 -0.515 -0.843]
 [ 4.181  3.37   2.823  2.276  1.729  1.182  0.634  0.087 -0.46  -0.815]
 [ 4.254  3.425  2.878  2.331  1.783  1.236  0.689  0.142 -0.405 -0.788]
 [ 4.342  3.48   2.932  2.385  1.838  1.291  0.744  0.197 -0.35  -0.761]
 [ 4.454  3.534  2.987  2.44   1.893  1.346  0.799  0.251 -0.296 -0.733]
 [ 4.609  3.589  3.042  2.495  1.948  1.4    0.853  0.306 -0.241 -0.706]
 [ 4.848  3.644  3.097  2.549  2.002  1.455  0.908  0.361 -0.186 -0.679]
 [ 5.3    3.698  3.151  2.604  2.057  1.51   0.963  0.416 -0.132 -0.651]] 

[[ 3.627  2.424  1.505  0.586 -0.334 -1.253 -2.172 -3.091 -4.01  -4.516]
 [ 3.705  2.516  1.597  0.677 -0.242 -1.161 -2.08  -2.999 -3.919 -4.47 ]
 [ 3.79   2.608  1.689  0.769 -0.15  -1.069 -1.988 -2.907 -3.827 -4.424]
 [ 3.887  2.7    1.781  0.861 -0.058 -0.977 -1.896 -2.815 -3.735 -4.378]
 [ 4.001  2.792  1.872  0.953  0.034 -0.885 -1.8

In [41]:
#create matrix compare fn with multiple use cases
def matrixcompare(m1, m2, operation):
    assert np.matrix(m1).shape == np.matrix(m2).shape, 'matrix size mismatch, cannot compare matrices of different sizes'
    lst1 = np.array(m1)
    lst2 = np.array(m2)
    output = np.zeros((len(lst1), len(lst1[0])))
    for i in range(0, len(lst1)):
        for j in range(0, len(lst1[0])):
            if operation == 'normeddiff':
                #calculate the difference between the mod and the average (how much bigger is the mod epv compared to average?)
                output[i][j] = lst2[i][j] - lst1[i][j]
            elif operation == 'combine':
                output[i][j] = (lst1[i][j] * lst2[i][j]) + lst2[i][j]
            else:
                break
        if operation == 'normeddiff':
            #normalize differences by row (normalized to be a percentage between - modweight and modweight)
            romi = -14 #these two values can be adjusted, but essentially they represent the max differences possible in 2 epvs. 
            romx = 14 #the most 2 epvs can be different is 14 or negative 14, which are the floor and ceiling of m_epv-a_epv
            output[i] = ((2 * (1 + modweight))*((np.array(output[i]) - romi) / (romx - romi)) - (1 + modweight))      
    return output

In [42]:
#compare mod epv with avg epv to create a percent of average epv that represents how much different the modded epv is from the norm
epv_diff = matrixcompare(a_epv10, m_epv10, 'normeddiff') #average EPV *FIRST*. 
print(epv_diff)
#multiply this with our average conversion percentages to modify conversion percentage
m_conv = matrixcompare(epv_diff, r_conv, 'combine')

[[-0.053 -0.102 -0.147 -0.192 -0.237 -0.282 -0.328 -0.373 -0.418 -0.443]
 [-0.05  -0.097 -0.142 -0.187 -0.233 -0.278 -0.323 -0.368 -0.413 -0.44 ]
 [-0.047 -0.093 -0.138 -0.183 -0.228 -0.273 -0.318 -0.364 -0.409 -0.438]
 [-0.044 -0.088 -0.133 -0.178 -0.224 -0.269 -0.314 -0.359 -0.404 -0.436]
 [-0.041 -0.084 -0.129 -0.174 -0.219 -0.264 -0.309 -0.355 -0.4   -0.434]
 [-0.038 -0.079 -0.124 -0.169 -0.215 -0.26  -0.305 -0.35  -0.395 -0.431]
 [-0.034 -0.075 -0.12  -0.165 -0.21  -0.255 -0.3   -0.346 -0.391 -0.429]
 [-0.029 -0.07  -0.115 -0.16  -0.206 -0.251 -0.296 -0.341 -0.386 -0.427]
 [-0.02  -0.065 -0.111 -0.156 -0.201 -0.246 -0.291 -0.337 -0.382 -0.425]]


In [43]:
r_conv = np.array(r_conv)
r_conv100 = pd.Series(np.repeat([j for i in r_conv for j in i ], 10))
m_conv = np.array(m_conv)
m_conv100 = pd.Series(np.repeat([j for i in m_conv for j in i ], 10))

In [44]:
#insert output statement here
np.savetxt("avg_conv_pct.csv", r_conv100, fmt = '%10.5f')
np.savetxt("mod_conv_pct_" + teamname + ".csv", m_conv100, fmt = '%10.5f')

In [45]:
#print values for testing purposes:

print('\n raw:\n')
print(r_conv)
print('\n adjusted:\n')
print(m_conv)


 raw:

[[0.704 0.696 0.711 0.713 0.725 0.73  0.739 0.735 0.752 0.739]
 [0.52  0.608 0.604 0.613 0.619 0.648 0.653 0.65  0.642 0.586]
 [0.415 0.537 0.531 0.536 0.541 0.58  0.579 0.577 0.558 0.469]
 [0.368 0.479 0.483 0.479 0.486 0.524 0.516 0.514 0.496 0.387]
 [0.359 0.435 0.453 0.437 0.447 0.479 0.466 0.464 0.45  0.335]
 [0.367 0.401 0.433 0.408 0.42  0.443 0.428 0.426 0.414 0.312]
 [0.371 0.376 0.416 0.388 0.398 0.413 0.403 0.401 0.384 0.314]
 [0.349 0.358 0.394 0.372 0.375 0.389 0.392 0.39  0.353 0.34 ]
 [0.282 0.344 0.358 0.358 0.346 0.367 0.394 0.394 0.316 0.386]]

 adjusted:

[[0.667 0.626 0.607 0.576 0.553 0.524 0.497 0.461 0.438 0.412]
 [0.494 0.549 0.518 0.498 0.475 0.468 0.442 0.411 0.376 0.328]
 [0.395 0.487 0.458 0.438 0.417 0.421 0.395 0.367 0.33  0.264]
 [0.352 0.437 0.419 0.393 0.377 0.383 0.354 0.33  0.295 0.218]
 [0.345 0.399 0.395 0.361 0.349 0.352 0.322 0.299 0.27  0.19 ]
 [0.353 0.369 0.379 0.339 0.33  0.328 0.298 0.277 0.25  0.177]
 [0.358 0.348 0.366 0.324 0.314 0